In [7]:
import pandas as pd
import requests
from plyer import notification
import datetime

In [8]:
#Criar a função de alerta de erro
def alerta():
    data_atual = datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")

    notification.notify(
        title= f"Erro {response.status_code}",
        message="Ocorreu uma falha no carregamentos dos dados da API" + "\nData atual: " + data_atual,
        app_name="Alerta",
        timeout=10)

In [9]:
#Consultar a API e gerar um dataframe com os feriados nacionais
ano = 2023
url =f"https://brasilapi.com.br/api/feriados/v1/{ano}"
response = requests.get(url)
if response.status_code == 200:
    data_json = response.json()
    feriados_nacionais = pd.DataFrame(data_json)
  
else:
    alerta()
#Alterar os nomes das colunas
feriados_nacionais = feriados_nacionais.rename(columns ={'date': 'Data', 'name': 'Feriado', 'type': 'Tipo'})
#Alterar o formato da data
feriados_nacionais['Data'] = pd.to_datetime(feriados_nacionais['Data']).dt.strftime('%d/%m/%Y')
#Deixar a coluna "Tipo" em português
feriados_nacionais['Tipo'] = feriados_nacionais['Tipo'].replace('national', 'Nacional')
#Consultar se existem valores nulos
feriados_nacionais.isna()
feriados_nacionais


,Data,Feriado,Tipo
0,01/01/2023,Confraternização mundial,Nacional
1,21/02/2023,Carnaval,Nacional
2,07/04/2023,Sexta-feira Santa,Nacional
3,09/04/2023,Páscoa,Nacional
4,21/04/2023,Tiradentes,Nacional
5,01/05/2023,Dia do trabalho,Nacional
6,08/06/2023,Corpus Christi,Nacional
7,07/09/2023,Independência do Brasil,Nacional
8,12/10/2023,Nossa Senhora Aparecida,Nacional
9,02/11/2023,Finados,Nacional


In [10]:
# Consultar a API e gerar um dataframe com as corretoras ativas no Brasil
url2 ="https://brasilapi.com.br/api/cvm/corretoras/v1"
response = requests.get(url2, timeout= 15)
if response.status_code == 200:
    data_json2 = response.json()
    corretoras = pd.DataFrame(data_json2)
else:
    alerta()
#Limpar o dataframe deixando apenas as colunas com os dados de contato
corretoras_ativas = corretoras.loc[corretoras['status'] == 'EM FUNCIONAMENTO NORMAL']
dados = ['nome_social', 'email', 'municipio', 'telefone']
corretoras_ativas = corretoras_ativas[dados]
#Alterar nomes das colunas
corretoras_ativas = corretoras_ativas.rename(columns ={'nome_social': 'Nome', 'email': 'E-mail', 'municipio': 'Município', 'telefone': 'Telefone'})
#Formatar a coluna 'Telefone' no formato 'xxxx-xxxx' para melhor visualização
corretoras_ativas['Telefone'] = corretoras_ativas['Telefone'].astype(str).str[:4] + '-' + corretoras_ativas['Telefone'].astype(str).str[4:]
#Capitalizar a primeira letra de cada palavra na coluna 'Município'
corretoras_ativas['Município'] = corretoras_ativas['Município'].str.title()
#Consultar se existem valores nulos
corretoras_ativas.isna()
corretoras_ativas

,Nome,E-mail,Município,Telefone
3,ABN AMRO CORRETORA DE T-TULOS E VALORES MOBILI...,canal.comunicacao@br.abnamro.com,São Paulo,3073-7417
7,"ACTIVTRADES CORRETORA DE CÂMBIO, TÍTULOS E VAL...",rramos@activtrades.com,Florianópolis,3054-2864
12,AGORA CTVM S.A.,ipc.supervisao.corretoras@bradesco.com.br,São Paulo,2178-6198
16,ALFA CORRETORA DE CAMBIO E VALORES MOBILIARIOS...,jelanir@bancoalfa.com.br,São Paulo,3175-5050
21,AMARIL FRANKLIN CTV LTDA,investimentos@amarilfranklin.com.br,Belo Horizonte,3235-8128
...,...,...,...,...
348,TORO CORRETORA DE TÍTULOS E VALORES MOBILIARIO...,juridico@toroinvestimentos.com.br,Belo Horizonte,3665-8700
355,TULLETT PREBON BRASIL CORRETORA DE VALORES E C...,juridico@tullettprebon.com.br,São Paulo,2111-4800
356,"UBS BRASIL CORRETORA DE CAMBIO, TITULOS E VALO...",daniel.bassan@ubs.com,São Paulo,2050-6500
370,WARREN CORRETORA DE VALORES MOBILIÁRIOS E CÂMB...,compliance@warren.com.br,Porto Alegre,3237-2273


In [11]:
#Consultar a API e gerar um dataframe com as taxas de juros vigentes no Brasil
url3 ="https://brasilapi.com.br/api/taxas/v1"
response = requests.get(url3)
if response.status_code == 200:
    data_json3 = response.json()
    taxas = pd.DataFrame(data_json3)
else:
    alerta()

#Alterar nomes das colunas
taxas = taxas.rename(columns= {'nome': 'Nome', 'valor': 'Valor (%)'})
#Consultar se existem valores nulos
taxas.isna()
taxas

,Nome,Valor (%)
0,Selic,12.75
1,CDI,12.65
2,IPCA,5.19


In [13]:
from sqlalchemy import create_engine

# Criar database
engine = create_engine('sqlite:///coderhouse.db')

# Importar dataframes para o database
feriados_nacionais.to_sql('feriados_nacionais', con=engine, if_exists='replace', index=False)
corretoras_ativas.to_sql('corretoras_ativas', con=engine, if_exists='replace', index=False)
taxas.to_sql('taxas', con=engine, if_exists='replace', index=False)

3